(5 pt) The research question is stated.  
(5 pt) Links to sources are provided and a small description about the data  
(20 pt) Data qualitity and data quantity are inspected and reported. Appropiate transformations are applied.  
(20 pt) Assumptions and presuppositions are made explicit (chosen data storage method, chosen analysis method, chosen design). An argumentative approach is used explaining steps, taken into account data quality and quantity. Explanation is provided either with comments in the code or in a seperate document.  
(10 pt) Interactive visualization is extracted from correct analysis of (incomplete) data  
(10 pt) The design supports the research question. The data is informative in relation to the topic. Visualization is functional and attractive Figures contain X and Y labels, title and captions. (10)  
(20 pt) Code is efficient coded, according to coding style without code smells and easy to read. Code is demonstrated robust and flexible  
(10 pt) All the code is stored in repository with Readme including most relevant information to implement the code. used software is suitably licensed and documented

# The effect of pets on psychological state during the pandemics

<b>Table of contents:</b>
1. [Data description](#description)
2. [Data inspection](#inspection)
    * [Data for Canada](#inspection_can)
    * [Data for Australia](#inspection_aus)
3. [Data analysis](#analysis)
    * [Canada data analysis](#analysis_can)
    * [Australia data analysis](#analysis_aus)
4. [Hypothesis testing](#hypothesis)
    * mental health of pet-owners is better than of those who don't have any pets
    * self-perceived mental health for Australians pet_owners differs from Canadians

## <a id='#description'>Data description</a>

* Results from survey completed by Canadian residents in Apr-Nov 2021.   
    Denis-Robichaud, Jose; Aenishaenslin, Cecile; Richard, Lucie; Desmarchelier, Marion; Carabin, Helene, 2021, "Association between Pet Ownership and Mental Health and Well-Being of Canadians Assessed in a Cross-Sectional Study during the COVID-19 Pandemic", https://doi.org/10.7910/DVN/JKDKWY, Harvard Dataverse, V5, UNF:6:7YJk+oBDFVpHnvlyBc1/Ew== [fileUNF]  
    The dataset includes the 23 variables used in the published manuscript.
    * D2: province or territory
    * region: Canadian geographical regions
    * O1: pet ownership
    * O8b: change in pet ownership
    * P1: pet attitude
    * D1: age
    * D5: highest level of education
    * D67: ethnicity
    * D4: gender
    * D11: number of people in the household
    * D8: yearly income
    * E3: change in income since the beginning of the pandemic
    * H2: disability
    * H1Ax: emotional, psychological or mental health condition
    * H4: change in mental health since the beginning of the pandemic
    * H56: tested positive to COVID-19 (or household member)
    * S: social support
    * QOL: quality of life (EQ-5D-5L score converted on the scale developed for Canada)
    * GH: self-assessed overall health
    * L1: loneliness
    * H3: perceived mental health
    * Q2: self-reported level of stress
    * Q1_cat: anxiety

*  Results from survey completed by majorly Australian residents in Jul-Aug 2020  
    Rohlf, Vanessa; Howell, Tiffani; Bennett, Pauleen (2021): Pandemic pets and people project. La Trobe. Dataset. https://doi.org/10.26181/60e7af7d87848 
    

## <a id='inspection'>Data inspection</a>

### <a id='inspection_can'>Data for Canada</a>

In [204]:
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter, Span,
                         Dropdown)
from bokeh.transform import transform
from bokeh.palettes import PuBuGn

from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown
from bokeh.plotting import curdoc

from scipy import stats as st
import warnings
warnings.filterwarnings("ignore")

output_notebook()

Loading BokehJS ...

In [205]:
def get_config():
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config
config = get_config()
db = config['db']

FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [ ]:
# Loading data from Canadian residents
try:
    data_can = pd.read_csv(db + 'dataset_petandcovid.csv', sep='\t', encoding='utf-8-sig')
except FileNotFoundError:
    print('No such file')

data_can.head()

In [ ]:
# Renaming columns according to readme file
data_can.columns = ['province', 'region', 'pet_owner', 'change_in_ownership', 'pet_attitude', 'age', 'educ_level', 
                'ethnicity', 'gender', 'household_people', 'income', 'income_change', 'disability', 'mental_health', 
                'change_in_mh', 'tested_positive', 'social_support', 'quality_of_life', 'health_self', 'loneliness', 
                'mh_self', 'stress_self', 'anxiety']
data_can.info()

In [ ]:
data_can['province'].value_counts()

In [ ]:
#correcting the encoding problem
data_can['province'] = data_can['province'].str.replace('\uFFFD', 'e')
data_can['province'].value_counts()

In [ ]:
data_can['region'] = data_can['region'].str.replace('\uFFFD', 'e')
data_can['region'].value_counts()

In [ ]:
data_can['pet_owner'].value_counts()

In [ ]:
data_can['change_in_ownership'].value_counts()

In [ ]:
# Excluding households which recently lost their pet to exclude effect on emotional state
data_can = data_can[data_can['change_in_ownership']!='Loss']
data_can.info()

In [ ]:
# Check for duplicates
print('Number of duplicates: {:.0f}'.format(data_can.duplicated().sum()))

Data was loaded, checked for quality and duplicates, the values are of correct data types.  
From 1500 entries in intial dataframe, 1327 records were extracted for further analysis.

### <a id='inspection_aus'>Data for Australia</a>

In [ ]:
# Loading data from Australian residents
try:
    data_aus = pd.read_csv(db + 'PandemicPetsAndPeopl.csv')
except FileNotFoundError:
    print('No such file')

data_aus.head()

In [ ]:
print('Number of people that started the survey: {:.0f}'.format(len(data_aus)))

In [ ]:
# Taking a look at the questions that people answered
for count, col in enumerate(data_aus.columns):
    print(count, col)

In [ ]:
# Leaving only several columns that can be used later for comparisson with Canadian residents and renaming them for convenience
data_aus = data_aus.iloc[:,[0, 2, 6, 7, 15, 25, 26, 27, 28, 56, 74, 132, 133, 135, 139]]
data_aus.columns = ['record_id', 'survey_time', 'has_cat', 'has_dog', 'no_pets', 'country', 'birth_year', 'gender', 'educ_level', 'tested_positive', 'loneliness', 'health_self', 'quality_of_life', 'mh_self', 'mh_prob_week']
data_aus.head()

In [ ]:
# Leaving in the table only residents from Australia
data_aus = data_aus[data_aus['country'] == 'Australia']
data_aus.info()

In [ ]:
# Check for duplicates
print('Number of duplicates: {:.0f}'.format(data_aus.duplicated().sum()))

In [ ]:
data_aus['no_pets'].value_counts()

It seems like the dataset has information only on pet-owners.

In [ ]:
# excluding households with pets other than cats or dogs
data_aus = data_aus[~((data_aus['has_cat'] == 'Unchecked')&(data_aus['has_dog'] == 'Unchecked')&(data_aus['no_pets'] == 'Unchecked'))]
data_aus.info()

Dealing with empty values:

In [ ]:
# For birth_year the option can be is to categorize people by age the same way it was done for Canadian data, 
# and 5 empty values can be left as 'not mentioned'
def age_category(row):
    if np.isnan(row['birth_year']):
        return('not mentioned')
    else:
        age = 2020 - row['birth_year']
        if age < 18:
            return '18-'
        if (age >= 18) and (age < 25):
            return '18-24'
        if (age >= 25) and (age < 35):
            return '25-34'
        if (age >= 35) and (age < 45):
            return '35-44'
        if (age >= 45) and (age < 55):
            return '45-54'
        if (age >= 55) and (age < 65):
            return '55-64'
        if age >= 65:
            return '65+'

data_aus['age'] = data_aus.apply(age_category, axis=1)
data_aus.drop('birth_year', axis=1, inplace=True)
data_aus.sample(5)

In [ ]:
# substituting empty values in gender column with 'not mentioned'
data_aus.loc[data_aus['gender'].isna(), 'gender'] = 'Other or did not answer'

In [ ]:
# loneliness
data_aus['loneliness'].value_counts()

In [ ]:
# Since the minimum score is 1, empty values can be substituted with zeros
data_aus.loc[data_aus['loneliness']=='Really does not describe my experience <br> 1', 'loneliness'] = 1
data_aus.loc[data_aus['loneliness']=='Really describes my experience <br> 5', 'loneliness'] = 5
data_aus.loc[data_aus['loneliness'].isna(), 'loneliness'] = 0
data_aus['loneliness'] = data_aus['loneliness'].astype('int')
data_aus['loneliness'].value_counts()

In [ ]:
data_aus.loc[data_aus['health_self'].isna(), 'health_self'] = 'unknown'
data_aus['health_self'].value_counts()

In [ ]:
data_aus.loc[data_aus['quality_of_life'].isna(), 'quality_of_life'] = 'unknown'
data_aus.loc[data_aus['mh_self'].isna(), 'mh_self'] = 'unknown'
data_aus.loc[data_aus['mh_prob_week'].isna(), 'mh_prob_week'] = 'unknown'

data_aus['mh_prob_week'].value_counts()

In [ ]:
# changing the type for survey timestamp
data_aus['survey_time'] = pd.to_datetime(data_aus['survey_time'], format='%d/%m/%Y %H:%M')
print(data_aus['survey_time'].min(), ' - ', data_aus['survey_time'].max())

In [ ]:
data_aus.info()

Data from Australians was converted to dataframe that would be easier to compare with Canadians. Column headers renamed for convenient use, data type corrected, checked for duplicates, n/a values were left in the dataframe but substituted with 'not mentioned'.  
Data available for period 16/Jul/2020 - 15/Jun/2021.  
From initial dataframe with 1162 entries, 836 entries were extracted by applying filters. 

## <a id='analysis'>Data analysis</a>

### <a id='analysis_can'>Canada data analysis</a>

In [ ]:
#pet-owners on the map
from folium import Map, Marker
from folium.plugins import MarkerCluster

canada_lat, canada_lng = 56.1304, -106.3468

m = Map(location=[canada_lat, canada_lng], zoom_start=4)
marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):       
    if row['province'] == 'Ontario':
        lat = 51.2538
        lng = -85.3232
    if row['province'] == 'Quebec':
        lat = 46.829853
        lng = -71.254028
    if row['province'] == 'British Columbia':
        lat = 53.7267
        lng = -127.6476
    if row['province'] == 'Alberta':
        lat = 53.9333
        lng = -116.5765
    if row['province'] == 'Saskatchewan':
        lat = 55.000000
        lng = -106.000000
    if row['province'] == 'Manitoba':
        lat = 56.415211
        lng = -98.739075
    if row['province'] == 'Nova Scotia':
        lat = 45.000000
        lng = -63.000000
    if row['province'] == 'New Brunswick':
        lat = 46.498390
        lng = -66.159668
    if row['province'] == 'Newfoundland and Labrador':
        lat = 53.135509
        lng = -57.660435
    if row['province'] == 'Yukon':
        lat = 64.000000
        lng = -135.000000
    if row['province'] == 'Prince Edward Island':
        lat = 46.250000
        lng = -63.000000
    if row['province'] == 'Northwest Territories':
        lat = 63.588753
        lng = -115.506938
    if row['province'] == 'Nunavut':
        lat = 70.453262
        lng = -86.798981
    Marker(
        [lat, lng],
    ).add_to(marker_cluster)

data_can[data_can['pet_owner']=='Yes'].apply(create_clusters, axis=1)

m

In [ ]:
# Distribution of people by age
grouped = data_can.groupby(['age', 'pet_owner'])['province'].count().reset_index()

plt.figure(figsize=(10, 7))
sns.barplot(x='age', y = 'province', data=grouped, hue='pet_owner', palette=['#7B8FA1', '#FAD6A5'])
plt.title('Number of people participated in the survey depending on their age')
plt.xlabel('Age group')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

Amount of people who are pet-owners prevails for age groups under 55 years. After 55 years there are more households without any pets, especially after age 65: amount of non pet-owners is almost double than of those with pet. The majority of pet-owners are aged 45-55.

In [ ]:
# Distribution of people by education level
grouped_ed = data_can.groupby(['educ_level', 'pet_owner'])['province'].count().reset_index()

plt.figure(figsize=(8, 5))
sns.barplot(x='educ_level', y = 'province', data=grouped_ed, hue='pet_owner', palette=['#7B8FA1', '#FAD6A5'])
plt.title('Number of people participated in the survey depending on their education level')
plt.xlabel('Education level')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

Number of people without any pet prevails only for people with university degree.

In [ ]:
# Distribution of people by income level
grouped_inc = data_can.groupby(['income', 'pet_owner'])['province'].count().reset_index()

plt.figure(figsize=(15, 8))
sns.barplot(x='income', y = 'province', data=grouped_inc, hue='pet_owner', palette=['#7B8FA1', '#FAD6A5'])
plt.title('Number of people participated in the survey depending on their income level')
plt.xlabel('Income level')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

Number of pet-owners is more among people either with low-income (<40k) or high-income (>100k). However, amount of households without pets is more among middle class (between 40k and 100k). 

In [ ]:
# Distribution of people by presense of mental health conditions
grouped_mh = data_can.groupby(['mental_health', 'pet_owner'])['province'].count().reset_index()

plt.figure(figsize=(8, 4))
sns.barplot(x='mental_health', y = 'province', data=grouped_mh, hue='pet_owner', palette=['#7B8FA1', '#FAD6A5'])
plt.title('Number of people participated in the survey depending on their mental health conditions')
plt.xlabel('Presense of mental health conditions')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

In [ ]:
# Distribution of people by presense of mental health conditions
grouped_mh = data_can.groupby(['change_in_mh', 'pet_owner'])['province'].count().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='change_in_mh', y = 'province', data=grouped_mh, hue='pet_owner', palette=['#7B8FA1', '#FAD6A5'])
plt.title('Number of people participated in the survey depending on change in mental health during the pandemic')
plt.xlabel('Change in mental health during the pandemic')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

The amount of people who felt worse during the pandemic is more for pet-owners.

In [ ]:
# Distribution of people by presense of mental health conditions
grouped_mh = data_can.groupby(['mh_self', 'pet_owner'])['province'].count().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='mh_self', y = 'province', data=grouped_mh, hue='pet_owner', palette=['#7B8FA1', '#FAD6A5'])
plt.title('Number of people participated in the survey depending on self perceived mental health')
plt.xlabel('Self perceived mental health')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

In [ ]:
# Distribution of people by presense of mental health conditions
grouped_mh = data_can.groupby(['anxiety', 'pet_owner'])['province'].count().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='anxiety', y = 'province', data=grouped_mh, hue='pet_owner', palette=['#7B8FA1', '#FAD6A5'])
plt.title('Number of people participated in the survey depending on self perceived mental health')
plt.xlabel('Self perceived mental health')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

Variable 'pet_owner' along with the 'mental_health', 'change_in_mh' and 'loneliness' are independent variables, so we cannot calculate correlation coefficient for them.

### <a id='analysis_aus'>Australia data analysis</a>

In [ ]:
# Distribution of people by age
grouped = data_aus.groupby(['age'])['record_id'].count().reset_index()

plt.figure(figsize=(10, 7))
sns.barplot(x='age', y = 'record_id', data=grouped, palette=['#FAD6A5'])
plt.title('Number of people participated in the survey depending on their age')
plt.xlabel('Age group')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

Among Australians the majority of pet-owners are aged 25-55 (for Canadians it was 45-55).

In [ ]:
# Distribution of people by loneliness
grouped_ed = data_aus.groupby(['loneliness'])['record_id'].count().reset_index()

plt.figure(figsize=(8, 5))
sns.barplot(x='loneliness', y = 'record_id', data=grouped_ed, palette=['#FAD6A5'])
plt.title('Number of people participated in the survey depending on loneliness')
plt.xlabel('Loneliness')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

The majority of Australian pet-owners didn't feel lonely during pandemic (score = '1'), but still that feeling wasn't uncommon among these people.

In [ ]:
# Distribution of people by loneliness
grouped_ed = data_aus.groupby(['mh_self'])['record_id'].count().reset_index()

plt.figure(figsize=(8, 5))
sns.barplot(x='mh_self', y = 'record_id', data=grouped_ed, palette=['#FAD6A5'])
plt.title('Number of people participated in the survey depending on self-perceived mental state')
plt.xlabel('Self-perceived mental state')
plt.ylabel('Number of people')
plt.grid(axis='y', alpha=0.2)
plt.show()

The majority of Australian pet-owners evaluate their mental state as 'Good' and 'Very good'. Which is almost the same for Canadian (where tha majority said that they had 'minimal' impact on their mental state).

## <a id='hypothesis'>Hypothesis testing</a>

<b>Change in mental health state for Canadian pet-owners differs from those who don't have any pets</b>

In [ ]:
pet_owners = data_can[data_can['pet_owner']=='Yes']
pet_owners['change_in_mh'] = pet_owners['change_in_mh'].map({'Much worse': -2, 'Somewhat worse': -1, 
                                                             'About the same': 0, 'Somewhat better': 1, 'Much better': 2})
not_pet_owners = data_can[data_can['pet_owner']=='No']
not_pet_owners['change_in_mh'] = not_pet_owners['change_in_mh'].map({'Much worse': -2, 'Somewhat worse': -1, 
                                                             'About the same': 0, 'Somewhat better': 1, 'Much better': 2})


In [ ]:
#Check for normal distribution: pet_owners
results = st.shapiro(pet_owners['change_in_mh'])
p_value = results[1]
alpha = 0.05
print('p-value: ', p_value)

if p_value < alpha:
    print('Distibution is not normal')
else:
    print('Distribution is normal')
    
sns.distplot(pet_owners['change_in_mh'])

In [ ]:
#Check for normal distribution: not_pet_owners
results = st.shapiro(not_pet_owners['change_in_mh'])
p_value = results[1]
alpha = 0.05
print('p-value: ', p_value)

if p_value < alpha:
    print('Distibution is not normal')
else:
    print('Distribution is normal')
    
sns.distplot(not_pet_owners['change_in_mh'])

Both distribution aren't normal based on the Shapio-Wilk test, we'll use Mann-Whitney non-parametric test to test the hypothesis:

In [ ]:
# H0: Change in mental health state for Canadian pet-owners is the same as for those who don't have any pets
# H1: Change in mental health state for Canadian pet-owners differs from those who don't have any pets

results = st.mannwhitneyu(pet_owners['change_in_mh'], not_pet_owners['change_in_mh'])

print('p-value: {:.2f}'.format(results.pvalue))

if results.pvalue < alpha:
    print('Rejecting null hypothesis, there is a statistically significant difference')
else:
    print('Null hypothesis cannot be rejected')

<b>Self-perceived mental health for Australians pet-owners differs from Canadians</b>

In [ ]:
pet_owners_can = data_can[data_can['pet_owner']=='Yes']
pet_owners_can['mh_self'] = pet_owners_can['mh_self'].map({'Poor': 0, 'Fair': 1, 'Good': 2, 
                                                           'Very good': 3, 'Excellent': 4})
pet_owners_aus = data_aus[data_aus['mh_self'] != 'unknown']
pet_owners_aus['mh_self'] = pet_owners_aus['mh_self'].map({'Poor': 0, 'Fair': 1, 'Good': 2, 
                                                           'Very good': 3, 'Excellent': 4})

In [ ]:
#Check for normal distribution: canadian pet_owners
results = st.shapiro(pet_owners_can['mh_self'])
p_value = results[1]
alpha = 0.05
print('p-value: {:.2f}'.format(p_value))

if p_value < alpha:
    print('Distibution is not normal')
else:
    print('Distribution is normal')
    
sns.distplot(pet_owners_can['mh_self'])

In [ ]:
#Check for normal distribution: australian pet_owners
results = st.shapiro(pet_owners_aus['mh_self'])
p_value = results[1]
alpha = 0.05
print('p-value: {:.2f}'.format(p_value))

if p_value < alpha:
    print('Distibution is not normal')
else:
    print('Distribution is normal')
    
sns.distplot(pet_owners_aus['mh_self'])

Both distribution aren't normal based on the Shapio-Wilk test, we'll use Mann-Whitney non-parametric test to test the hypothesis:

In [ ]:
# H0: Self-perceived mental health for Australian pet-owners and Canadian is the same
# H1: Self-perceived mental health for Australian pet-owners differs from Canadian

results = st.mannwhitneyu(pet_owners_aus['mh_self'], pet_owners_can['mh_self'])

print('p-value: {:.2f}'.format(results.pvalue))

if results.pvalue < alpha:
    print('Rejecting null hypothesis, there is a statistically significant difference')
else:
    print('Null hypothesis cannot be rejected')

<b>Conclusion</b>  
The data from Canadian and Australian residents was analysed. However, only for Canadians it was possible to compare the effect of pets, due to the reason that in Australia only pet-owners completed the survey. Suprisingly, the amount of people who felt worse during the pandemic is more among pet-owners, but we cannot be sure that there is a direct correlation as there can be another factors affecting (for example people could had been already diagnosed with some mental health diseases, which got worsened by self isolation).  
Conducting statistical testing of the hypothesis we can say that there is no statistical difference between self-assessed mental state of pet-owners in Australia and in Canada.  
For more clear results there should be more data and different factors should be taken into account.

In [132]:
from bokeh.models import Dropdown

menu_widget = [('Age', 'age'), ('Educational level', 'educ_level'), ('Gender', 'gender'), ('Income', 'income'), ('Mental health', 'mental_health'),
              ('Change in mental health', 'change_in_mh'), ('Loneliness', 'loneliness'), ('Anxiety', 'anxiety')]
menu_dropdown = Dropdown(label = "Filter by", menu = menu_widget)
show(menu_dropdown)

In [135]:
def callback(self, attr, old, new):
    subset = data_can.groupby([menu_dropdown.value, 'pet_owner'])['province'].count().reset_index()
    #p = Bar(subset, title="Number of people participated in the survey depending on", xlabel=menu_dropdown.value, ylabel='Number of people', width=400, height=400)
    
    p = figure(x_range=subset, plot_height=400, plot_width=400, 
        title="Number of people",
        toolbar_location=None, tools="")
    p.vbar(x=menu_dropdown.value, top='province', width=0.9, source=subset, line_color="white",
       # use the palette to colormap based on the the x[1:2] values
       fill_color=factor_cmap('x', palette=palette, factors='pet_owner', start=1, end=2))
    
    output_server()    
    show(p)

print(menu_dropdown.value)
menu_dropdown.on_change('value', self.callback)
curdoc().add_root(menu_dropdown)

AttributeError: unexpected attribute 'value' to Dropdown, possible attributes are align, aspect_ratio, background, button_type, css_classes, default_size, disabled, height, height_policy, icon, js_event_callbacks, js_property_callbacks, label, margin, max_height, max_width, menu, min_height, min_width, name, sizing_mode, split, subscribed_events, syncable, tags, visible, width or width_policy